In [13]:
#Preparación de librerias
import pandas as pd
import numpy as np

In [14]:
#Lectura de datos de formato csv
tdf = pd.read_csv('../DATOS/Toyota.csv')

In [15]:
#Definición de la función estado
def estado(cambio):
    if cambio > -0.4 and cambio < 0.4 :
        return 'Estable'
    elif cambio < 0:
        return 'Bajo'
    else:
        return 'Subio'
    
#Se asigna un estado ('Estable', 'Bajo', 'Subio') dependiendo del valor de cambio

#Calcula la diferencia de los valores de cierre para obtener el cambio
tdf['Cambio'] = tdf['Close'].diff()

#Aplica la función 'estado' a cada valor de cambio para asignar un estado
tdf['EstadoNuevo'] = tdf['Cambio'].apply(estado)

#Calcula la matriz de transición entre los estados y normaliza los valores para obtener probabilidades
#Aqui los datos estan normalizados
transicion = pd.crosstab(tdf['EstadoNuevo'].shift(), tdf['EstadoNuevo'], normalize=True)

#Nivel de confianza que se usa para la prueba de chi cuadrado
nivelConfianza = 0.95

#Calcula el valor de alpha, que seria el nivel de significancia para la prueba de hipotesis
alpha = 1 - nivelConfianza #Seria 0.05

#Función para calcular chi cuadrado
def chi2 (transicion):
    frecuencias = transicion.values
    fsum = transicion.sum(axis = 1)
    csum = transicion.sum(axis = 0)
    tsum = transicion.sum().sum()
    frecuencias_esperadas = np.outer(fsum, csum) / tsum
    observado_chi = np.sum((frecuencias - frecuencias_esperadas)**2/frecuencias_esperadas)
    grados = (transicion.shape[0]-1) * (transicion.shape[1]-1)
    return observado_chi, grados

#Calcula la prueba de chi cuadrado para la matriz de transición

#Valor citico para la prueba de chi cuadrado
valorCritico = 5.9915

#Calcula chi cuadrado y evalua si se rechaza la hipotesis nula
observado_chi, grados = chi2(transicion)
print(f"Chi Observado: {observado_chi}")
print(f"\nGrados de libertad: {grados}")
print(f"Teniendo en cuenta que nuestros grados de libertad son {grados} y tenemos un valor critico de {valorCritico} entonces:")
if observado_chi > valorCritico:
    print("Se rechaza")
else:
    print("No se rechaza")

print(transicion)

#Verifica la suma de probabilidades por estado
for estado in transicion.index:
    suma_probabilidades = transicion.loc[estado].sum()
    print(f"Suma de probabilidades para el estado {estado}: {suma_probabilidades}")

#Calcula y muestra las probabilidades del siguiente estado para cada semana
for semanaActual in range(len(tdf)):
    actual = tdf['EstadoNuevo'].iloc[semanaActual]
    psiguiente = transicion.loc[actual]
    psiguienteN = psiguiente / psiguiente.sum()

    print(f"\nSemana {semanaActual + 1}")
    print(f"\nEstado actual - {actual}")
    print(f"\nProbabilidad siguiente: {psiguienteN}")

Chi Observado: 0.0008656290837568107

Grados de libertad: 4
Teniendo en cuenta que nuestros grados de libertad son 4 y tenemos un valor critico de 5.9915 entonces:
No se rechaza
EstadoNuevo      Bajo   Estable     Subio
EstadoNuevo                              
Bajo         0.196154  0.042308  0.200000
Estable      0.042308  0.011538  0.050000
Subio        0.200000  0.050000  0.207692
Suma de probabilidades para el estado Bajo: 0.43846153846153846
Suma de probabilidades para el estado Estable: 0.10384615384615385
Suma de probabilidades para el estado Subio: 0.45769230769230773

Semana 1

Estado actual - Subio

Probabilidad siguiente: EstadoNuevo
Bajo       0.436975
Estable    0.109244
Subio      0.453782
Name: Subio, dtype: float64

Semana 2

Estado actual - Subio

Probabilidad siguiente: EstadoNuevo
Bajo       0.436975
Estable    0.109244
Subio      0.453782
Name: Subio, dtype: float64

Semana 3

Estado actual - Bajo

Probabilidad siguiente: EstadoNuevo
Bajo       0.447368
Estable    

In [16]:
def es_estacionaria(transicion):
    for estado in transicion.index:
        suma_probabilidades_estacionaria = transicion.loc[estado].sum()
        print(f"Suma de probabilidades para el estado {estado}: {suma_probabilidades_estacionaria}")
        if abs(suma_probabilidades_estacionaria - 1) > 0.05:
            return False
    return True

def verificar_estacionariedad(tdf, transicion):
    transiciones = {}
    for semanaActual in range(1, len(tdf)):
        estado_anterior = tdf['EstadoNuevo'].iloc[semanaActual - 1]
        estado_actual = tdf['EstadoNuevo'].iloc[semanaActual]

        if estado_anterior not in transiciones:
            transiciones[estado_anterior] = {}
        
        if estado_actual not in transiciones[estado_anterior]:
            transiciones[estado_anterior][estado_actual] = 0
        
        transiciones[estado_anterior][estado_actual] += 1

    total_transiciones = sum(sum(transiciones[estado].values()) for estado in transiciones)
    for estado in transiciones:
        for estado_siguiente in transiciones[estado]:
            transiciones[estado][estado_siguiente] /= total_transiciones
    
    # Verificar si las probabilidades de transición son constantes
    for i in range(1, len(tdf)):
        estado_anterior = tdf['EstadoNuevo'].iloc[i - 1]
        estado_actual = tdf['EstadoNuevo'].iloc[i]
        
        probabilidad_transicion = transiciones.get(estado_anterior, {}).get(estado_actual, 0)
        
        if probabilidad_transicion == 0:
            print(f"La probabilidad de transición de {estado_anterior} a {estado_actual} es 0 en la semana {i+1}.")
            return False
    return True

if es_estacionaria(transicion):
    print("La matriz de transición es estacionaria")
else:
    print("La matriz de transición no es estacionaria")

if verificar_estacionariedad(tdf, transicion):
    print("La matriz de transición es estacionaria en todas las semanas")
else:
    print("La matriz de transición no es estacionaria en todas las semanas")

Suma de probabilidades para el estado Bajo: 0.43846153846153846
La matriz de transición no es estacionaria
La matriz de transición es estacionaria en todas las semanas


In [17]:
import numpy as np
# estacionariedad

tdf['EstadoNuevo_actual'] = tdf['EstadoNuevo']
tdf['EstadoNuevo_siguiente'] = tdf['EstadoNuevo'].shift(-1)

num_periodos = 10

periodos = np.array_split(tdf, num_periodos)

for i, periodo in enumerate(periodos, start=1):
    periodo['transición'] = periodo['EstadoNuevo_actual'] + ' -> ' + periodo['EstadoNuevo_siguiente']
    conteo_transiciones = periodo['transición'].value_counts()
    total_transiciones = conteo_transiciones.sum()
    print(f"\nPeriodo {i}")
    print(conteo_transiciones)
    print(f"Total de transiciones: {total_transiciones}")


Periodo 1
transición
Subio -> Subio        7
Subio -> Bajo         5
Bajo -> Subio         4
Bajo -> Estable       3
Estable -> Bajo       3
Bajo -> Bajo          2
Subio -> Estable      1
Estable -> Subio      1
Estable -> Estable    1
Name: count, dtype: int64
Total de transiciones: 27

Periodo 2
transición
Bajo -> Subio         4
Subio -> Subio        4
Bajo -> Bajo          4
Subio -> Estable      3
Estable -> Bajo       3
Subio -> Bajo         3
Estable -> Subio      2
Bajo -> Estable       2
Estable -> Estable    1
Name: count, dtype: int64
Total de transiciones: 26

Periodo 3
transición
Bajo -> Bajo        5
Subio -> Bajo       5
Bajo -> Subio       5
Subio -> Subio      4
Bajo -> Estable     3
Estable -> Bajo     2
Estable -> Subio    1
Subio -> Estable    1
Name: count, dtype: int64
Total de transiciones: 26

Periodo 4
transición
Bajo -> Bajo          6
Bajo -> Subio         6
Subio -> Bajo         6
Subio -> Subio        4
Estable -> Bajo       2
Estable -> Estable    1
Bajo

c:\Users\Deutsch\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [18]:
# finding the stationary distribution of the Markov chain

from statsmodels.tsa.stattools import adfuller

def es_estacionaria(serie, umbral=0.05):
    p_valor = adfuller(serie)[1]
    return p_valor < umbral

conteos=[]
estacionarias = []
no_estacionarias = []

for i, periodo in enumerate(periodos, start=1):
    transiciones = periodo['transición'].value_counts()
    conteos.append(transiciones)
    print(conteos)

    if es_estacionaria(transiciones.values.ravel()):
        estacionarias.append(i)
    else:
        no_estacionarias.append(i)
   
#print(f"\nPeriodos estacionarios: {estacionarias}")
#print(f"Periodos no estacionarios: {no_estacionarias}")


ModuleNotFoundError: No module named 'statsmodels'